In [1]:
import numpy as np
from typing import List
import torch

from lib.Tensor import Tensor, force_tensor_method
from lib.NN import Dense, ReLU, Module

In [2]:
class MLP(Module):
    def __init__(self, layer_dims: List[int], activation_fn=ReLU) -> None:
        super().__init__()

        for i in range(len(layer_dims) - 1):
            self.add_module(f"dense_{i}", Dense(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:
                self.add_module(f"activation_{i}", activation_fn())

    @force_tensor_method
    def forward(self, x: Tensor) -> Tensor:
        for name, module in self._modules.items():
            x = module(x)
        return x

In [3]:
layer_dims = [2, 4, 4, 1]
model = MLP(layer_dims)
input = Tensor(np.random.randn(10, 2), requires_grad=True)

output = model(input)
output

Tensor([[-9.48008600e-06]
 [-6.44903064e-06]
 [-1.06699067e-07]
 [-2.05223748e-06]
 [-1.12727728e-06]
 [-5.42491098e-06]
 [-4.97322266e-06]
 [-2.34052912e-06]
 [-9.24606180e-07]
 [-2.70473034e-08]], requires_grad=True)

In [4]:
model.list_modules()

['dense_0', 'activation_0', 'dense_1', 'activation_1', 'dense_2']

In [5]:
model.get_module("dense_1").parameters()

[Tensor([[ 0.0022283  -0.0096725  -0.00256884 -0.0070397 ]
  [ 0.00852427  0.01372309  0.0135955   0.00093445]
  [-0.00859275  0.01060837  0.01039665  0.00204073]
  [-0.00146119  0.0084003  -0.01015021 -0.01345416]], requires_grad=True),
 Tensor([0. 0. 0. 0.], requires_grad=True)]

In [6]:
input_dim = 5
hidden_dim = 10
output_dim = 2
batch_size = 32

layer_dims = [input_dim, hidden_dim, output_dim]

# Create your custom MLP and PyTorch MLP
mlp = MLP(layer_dims, activation_fn=ReLU)
pt_mlp = torch.nn.Sequential(
    torch.nn.Linear(input_dim, hidden_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_dim, output_dim)
)

with torch.no_grad():
    my_dense_0_params = mlp.get_module("dense_0").parameters()
    pt_mlp[0].weight.copy_(torch.tensor(my_dense_0_params[0].data))
    pt_mlp[0].bias.copy_(torch.tensor(my_dense_0_params[1].data))

    my_dense_1_params = mlp.get_module("dense_1").parameters()
    pt_mlp[2].weight.copy_(torch.tensor(my_dense_1_params[0].data))
    pt_mlp[2].bias.copy_(torch.tensor(my_dense_1_params[1].data))

    # Create random input tensor
    pt_input = torch.randn(batch_size, input_dim)
    my_input = Tensor(pt_input.numpy())  # Assuming you have a way to convert numpy arrays to your Tensor class

    # Forward pass
    pt_output = pt_mlp(pt_input)
    my_output = mlp(my_input)

    print(f"My output: {my_output.data[:2]}")
    print(f"PyTorch output: {pt_output[:2]}")

    assert np.allclose(my_output.data, pt_output.detach().numpy(), atol=1e-6)

My output: [[ 0.00034548 -0.0004273 ]
 [ 0.00012918 -0.00039618]]
PyTorch output: tensor([[ 0.0003, -0.0004],
        [ 0.0001, -0.0004]])


t

In [7]:
def test_mlp():
    input_dim = 5
    hidden_dim = 10
    output_dim = 2
    batch_size = 32

    layer_dims = [input_dim, hidden_dim, output_dim]

    # Create your custom MLP and PyTorch MLP
    mlp = MLP(layer_dims, activation_fn=ReLU)
    pt_mlp = torch.nn.Sequential(
        torch.nn.Linear(input_dim, hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(hidden_dim, output_dim)
    )

    with torch.no_grad():
        my_dense_0_params = mlp.get_module("dense_0").parameters()
        pt_mlp[0].weight.copy_(torch.tensor(my_dense_0_params[0].data))
        pt_mlp[0].bias.copy_(torch.tensor(my_dense_0_params[1].data))

        my_dense_1_params = mlp.get_module("dense_1").parameters()
        pt_mlp[2].weight.copy_(torch.tensor(my_dense_1_params[0].data))
        pt_mlp[2].bias.copy_(torch.tensor(my_dense_1_params[1].data))

        # Create random input tensor
        pt_input = torch.randn(batch_size, input_dim)
        my_input = Tensor(pt_input.numpy())  # Assuming you have a way to convert numpy arrays to your Tensor class

        # Forward pass
        pt_output = pt_mlp(pt_input)
        my_output = mlp(my_input)

        # print(f"My output: {my_output.data[:2]}")
        # print(f"PyTorch output: {pt_output[:2]}")

        assert np.allclose(my_output.data, pt_output.detach().numpy(), atol=1e-6)

test_mlp()